# Basic Features of typed-polars

In [1]:
from tpolars import TypedFrame
from pydantic import BaseModel

### Create a simple pydantic model

In [2]:
class Cat(BaseModel):
    name: str
    age: int

### Instantiate a typed frame using the pydantic model as a generic argument

In [3]:
tf = TypedFrame[Cat]({"name": ["marty", "linus", "oscar"], "age": [14, 12, 2]})

The column schema is the pydantic model we specified

In [4]:
tf.col_schema

__main__.Cat

The pydantic attributes can be accessed, and return polars Expr objects that can be used in select statements 

In [5]:
tf.name

<Expr ['col("name")'] at 0x7FE2153194F0>

In [6]:
tf.select(tf.name)

name
str
"""marty"""
"""linus"""
"""oscar"""


### Get pydantic objects back out of the dataframe

In [7]:
tf[0]

Cat(name='marty', age=14)

## Comparison to raw polars dataframes

In [8]:
import polars as pl

df = pl.DataFrame({"name": ["marty", "linus", "oscar"], "age": [14, 12, 2]})

# annoying to type pl.col everywhere and put my column names in string quotes
df.select(pl.col("name"))

name
str
"""marty"""
"""linus"""
"""oscar"""


Ok, maybe this isn't too terrible (I definitely dislike typing out my columns in string quotes, and repeating pl.col everywhere). But this is a simple example, what if our query is more complicated?

In [11]:
df.select(pl.concat_str(pl.col("name"), pl.col("age"), separator=": ").alias("combined"))

combined
str
"""marty: 14"""
"""linus: 12"""
"""oscar: 2"""


In [12]:
tf.select(pl.concat_str(tf.name, tf.age, separator=": ").alias("combined"))

combined
str
"""marty: 14"""
"""linus: 12"""
"""oscar: 2"""


Starting to make TypedFrames look a bit better! Now imagine you have a dataframe with 20 columns, and you want to transform 5 of them into a derived feature, then do similar operations for 5 more features, and you can see how many times you repeat `pl.col(...)`. Also, I will eventually figure out how to make this compatible with static type checkers, which will really improve the experience. I can't count how many times I've tried to a complex polars query only to find `pl.col("col_name_with_typo")`